# Stokes Cavity Problem (3D): block solver
This notebook implements a linear block solver for the 3D Stokes cavity problem.
\begin{equation}
\left\{
    \begin{array}{ll}
        -\nu\Delta \mathbf{u}+\nabla p = \mathbf{f} & \mathbf{x}\in\Omega\\
        \nabla\cdot \mathbf{u} = 0 & \mathbf{x}\in\Omega\\
        \mathbf{u} = \mathbf{u}_{D} & \mathbf{x}\in\Gamma_{lid}\\
        \mathbf{u} = \mathbf{0} &\mathbf{x}\in\partial \Omega/\Gamma_{lid}
    \end{array}
\right.
\end{equation}

In [1]:
import dolfinx
from dolfinx import fem, mesh

from dolfinx.fem import FunctionSpace, Function
from mpi4py import MPI
from petsc4py import PETSc
from dolfinx.io import gmshio, XDMFFile

import basix, basix.ufl_wrapper
import matplotlib.pylab as plt
import numpy as np

from ufl import (VectorElement, FiniteElement,
                 SpatialCoordinate, TrialFunction, TestFunction,
                 as_vector, cos, sin, inner, div, grad, dx, pi)

N = 40
domain = dolfinx.mesh.create_unit_cube(MPI.COMM_WORLD, N, N, N,
                                     cell_type=dolfinx.mesh.CellType.tetrahedron)
P2 = VectorElement("Lagrange", domain.ufl_cell(), 2)
P1 = FiniteElement("Lagrange", domain.ufl_cell(), 1)

P1b = VectorElement(FiniteElement("Lagrange", domain.ufl_cell(), 1) 
                        + FiniteElement("Bubble", domain.ufl_cell(), 4))

V = fem.FunctionSpace(domain, P1b)
Q = fem.FunctionSpace(domain, P1)

def right(x, tol=1e-14): return x[0] > 1 - tol


def top_bottom(x, tol=1e-14):
    return np.logical_or(
        np.isclose(x[1], 0, atol=tol), np.isclose(x[1], 1, atol=tol))

def inflow(x):
    zero_comp = np.zeros(x.shape[1], dtype=np.float64)
    return (-np.sin(np.pi*x[1]), zero_comp, zero_comp)

def create_BC(V, domain):
    nonslip = fem.Constant(domain, [0.0, 0.0, 0.0])
    nonslip_facets = dolfinx.mesh.locate_entities_boundary(domain, domain.topology.dim-1, top_bottom)
    nonslip_dofs = dolfinx.fem.locate_dofs_topological(V, domain.topology.dim-1, nonslip_facets)
    bc0 = dolfinx.fem.dirichletbc(nonslip, nonslip_dofs, V)

    inflow_facets = dolfinx.mesh.locate_entities_boundary(domain, domain.topology.dim-1, right)
    inflow_dofs = dolfinx.fem.locate_dofs_topological(V, domain.topology.dim-1, inflow_facets)
    u_bc = dolfinx.fem.Function(V)
    u_bc.interpolate(inflow)
    bc1 = dolfinx.fem.dirichletbc(u_bc, inflow_dofs)

    return [bc0, bc1]

BC = create_BC(V, domain)

The variational formulation is the following
\begin{equation}
\nu\int_\Omega \nabla\mathbf{u}\cdot \nabla \mathbf{v}\,d\Omega -\int_\Omega p \, (\nabla \cdot \mathbf{v})\,d\Omega + \int_\Omega q \, (\nabla \cdot \mathbf{u})\,d\Omega=0 \qquad \forall \mathbf{v}\in\mathcal{V}_0, q\in\mathcal{Q}
\end{equation}

In [2]:
# This is needed since the pressure is defined up to a constant
def create_nullspace(rhs_form):
    null_vec = fem.petsc.create_vector_nest(rhs_form)
    null_vecs = null_vec.getNestSubVecs()
    null_vecs[0].set(0.0)
    null_vecs[1].set(1.0)
    null_vec.normalize()
    nsp = PETSc.NullSpace().create(vectors=[null_vec])
    return nsp

class stokes():
    def __init__(self, domain, V, Q, nu, BC):

        self.domain = domain
        self.V = V
        self.Q = Q
        self.nu = nu
        self.bc = BC

        self.u, self.p = TrialFunction(self.V), TrialFunction(self.Q)
        self.v, self.q = TestFunction(self.V), TestFunction(self.Q)
        
        self.f = fem.Constant(domain, (0., 0., 0.))
        self.g = fem.Constant(domain, 0.)

    def assemble(self):
        self.a_uu = self.nu * inner(grad(self.u), grad(self.v)) * dx 
        self.a_up = inner(-self.p, div(self.v)) * dx
        self.a_pu = inner(-self.q,  div(self.u)) * dx

        self.a = fem.form([[self.a_uu, self.a_up],[self.a_pu, None]])

        self.L_u = inner(self.f, self.v) * dx
        self.L_p = inner(self.g, self.q) * dx

        self.L = fem.form([self.L_u,
                           self.L_p])

        self.nsp = create_nullspace(self.L)

        self.A = fem.petsc.assemble_matrix_nest(self.a, bcs=self.bc)
        self.A.assemble()

        self.A.setNullSpace(self.nsp)

        self.b = fem.petsc.assemble_vector_nest(self.L)


        # Preconditioner 
        self.a_p11 = fem.form(inner(self.p, self.q) * dx)
        self.a_p = fem.form([[self.a[0][0], None],
                        [None, self.a_p11]])
        self.P = fem.petsc.assemble_matrix_nest(self.a_p, self.bc)
        self.P.assemble()   

        # Solver
        self.ksp = PETSc.KSP().create(self.domain.comm)
        self.ksp.setOperators(self.A, self.P)
        self.ksp.setTolerances(rtol=1e-9)
        self.ksp.getPC().setType("fieldsplit")
        self.ksp.getPC().setFieldSplitType(PETSc.PC.CompositeType.ADDITIVE)

        self.nested_IS = self.P.getNestISs()
        self.ksp.getPC().setFieldSplitIS(("u", self.nested_IS[0][0]),
                                         ("p", self.nested_IS[0][1]))

        # Set the preconditioners for each block
        self.ksp_u, self.ksp_p = self.ksp.getPC().getFieldSplitSubKSP()
        self.ksp_u.setType(PETSc.KSP.Type.BCGS)
        self.ksp_u.getPC().setType("gamg")
        self.ksp_p.setType(PETSc.KSP.Type.BCGS)
        self.ksp_p.getPC().setType("jacobi")

        # Monitor the convergence of the KSP
        self.ksp.setFromOptions()
        
    def solve(self):

        fem.petsc.apply_lifting_nest(self.b, self.a, bcs=self.bc)
        for b_sub in self.b.getNestSubVecs():
            b_sub.ghostUpdate(addv=PETSc.InsertMode.ADD,
                            mode=PETSc.ScatterMode.REVERSE)
        self.spaces = fem.extract_function_spaces(self.L)
        self.bcs0 = fem.bcs_by_block(self.spaces, self.bc)
        fem.petsc.set_bc_nest(self.b, self.bcs0)

        sol_u, sol_p = fem.Function(self.V), fem.Function(self.Q)
        w = PETSc.Vec().createNest([sol_u.vector, sol_p.vector])
        self.ksp.solve(self.b, w)

        sol_u.x.scatter_forward()
        sol_p.x.scatter_forward()

        return sol_u, sol_p

    def save(self, u, p, filename):
        
        u.name = "u"
        with XDMFFile(MPI.COMM_WORLD, filename+"_u.xdmf", "w") as xdmf:
          xdmf.write_mesh(domain)
          xdmf.write_function(u)

        p.name = "p"
        with XDMFFile(MPI.COMM_WORLD, filename+"_p.xdmf", "w") as xdmf:
          xdmf.write_mesh(domain)
          xdmf.write_function(p)

In [3]:
StokesPb = stokes(domain, V, Q, 1e-2, BC)
StokesPb.assemble()

In [ ]:
sol_u, sol_p = StokesPb.solve()
StokesPb.save(sol_u, sol_p, 'stokes')